In [2]:
import os
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split, KFold
from tensorflow.keras.metrics import Precision, Recall, AUC
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.regularizers import l2
import pydicom
from PIL import Image

In [3]:
def load_dicom_image(path, img_size=256):
    # Load dicom and convert to 256x256 RGB image
    dicom = pydicom.dcmread(path)
    image = dicom.pixel_array
    image = Image.fromarray(image).convert('RGB')
    image = image.resize((img_size, img_size))
    image = np.array(image) / 255.0

    # Convert to TensorFlow tensor
    print(f"Loaded image shape: {image.shape}, dtype: {image.dtype}")
    return tf.convert_to_tensor(image, dtype=tf.float32)

In [4]:
def process_directory(directory_path, label, img_size=256):
    images, labels = [], []
    for root, dirs, files in os.walk(directory_path):
        for filename in files:
            # Check if the file is a DICOM file
            if filename.endswith('.dcm'):
                # Construct the full path to the DICOM file
                file_path = os.path.join(root, filename)
                # Load and process the image
                image = load_dicom_image(file_path, img_size=img_size)
                # Append the processed image and its label to the lists
                images.append(image)
                labels.append(label)
    return images, labels

In [5]:
def create_datasets(cancer_dir, non_cancer_dir, img_size=256):
    # Load and process images and labels
    cancer_images, cancer_labels = process_directory(cancer_dir, 1, img_size=img_size)
    non_cancer_images, non_cancer_labels = process_directory(non_cancer_dir, 0, img_size=img_size)
    
    # Combine images and labels
    all_images = np.concatenate([np.array(cancer_images), np.array(non_cancer_images)], axis=0)
    all_labels = np.array(cancer_labels + non_cancer_labels)

    # After combining images and labels
    print(f"all_images shape: {all_images.shape}, dtype: {all_images.dtype}")
    print(f"all_labels shape: {all_labels.shape}, dtype: {all_labels.dtype}")

    
    return all_images, all_labels

In [7]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal_and_vertical"),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomZoom(0.1),
    tf.keras.layers.RandomTranslation(height_factor=0.1, width_factor=0.1)
])


2024-04-10 12:34:28.271285: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Max
2024-04-10 12:34:28.271315: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 48.00 GB
2024-04-10 12:34:28.271320: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 18.00 GB
2024-04-10 12:34:28.271535: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-04-10 12:34:28.271553: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [10]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.regularizers import l2

def build_model_with_resnet(input_shape=(256, 256, 3), dropout_rate=0.5, l2_reg=0.001):
    base_model = ResNet50(input_shape=input_shape,
                          include_top=False,
                          weights='imagenet')
    base_model.trainable = False  

    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dropout(dropout_rate),
        Dense(1, activation='sigmoid', kernel_regularizer=l2(l2_reg))
    ])
    
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    
    return model


In [11]:
# Load all data
all_images, all_labels = create_datasets(
    cancer_dir='/Users/arjunmoorthy/Desktop/Research_Capstone/ImageData/CapstoneData/cancer',
    non_cancer_dir='/Users/arjunmoorthy/Desktop/Research_Capstone/ImageData/CapstoneData/non_cancer',
    img_size=256)

# Split data into training+validation and test sets
(train_val_images, test_images, train_val_labels, test_labels) = train_test_split(
    all_images, all_labels, test_size=0.2, random_state=42)

# Prepare for 5-Fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

Loaded image shape: (256, 256, 3), dtype: float64
Loaded image shape: (256, 256, 3), dtype: float64
Loaded image shape: (256, 256, 3), dtype: float64
Loaded image shape: (256, 256, 3), dtype: float64
Loaded image shape: (256, 256, 3), dtype: float64
Loaded image shape: (256, 256, 3), dtype: float64
Loaded image shape: (256, 256, 3), dtype: float64
Loaded image shape: (256, 256, 3), dtype: float64
Loaded image shape: (256, 256, 3), dtype: float64
Loaded image shape: (256, 256, 3), dtype: float64
Loaded image shape: (256, 256, 3), dtype: float64
Loaded image shape: (256, 256, 3), dtype: float64
Loaded image shape: (256, 256, 3), dtype: float64
Loaded image shape: (256, 256, 3), dtype: float64
Loaded image shape: (256, 256, 3), dtype: float64
Loaded image shape: (256, 256, 3), dtype: float64
Loaded image shape: (256, 256, 3), dtype: float64
Loaded image shape: (256, 256, 3), dtype: float64
Loaded image shape: (256, 256, 3), dtype: float64
Loaded image shape: (256, 256, 3), dtype: float64


In [14]:
def preprocess(image, label):
    # Apply the augmentation to the image.
    image = data_augmentation(image)
    return image, label

# Assuming you have training images and labels loaded as NumPy arrays
train_dataset = tf.data.Dataset.from_tensor_slices((train_val_images, train_val_labels))
train_dataset = train_dataset.map(preprocess).batch(32).prefetch(tf.data.AUTOTUNE)

In [ ]:
# without data augmentation
""" # Cross-validation loop
for fold, (train_idx, val_idx) in enumerate(kf.split(train_val_images)):
    # Example for checking before creating a dataset for a fold
    print(f"Training dataset shape: {train_val_images[train_idx].shape}")
    print(f"Training labels shape: {train_val_labels[train_idx].shape}")
    print(f"Training on fold {fold+1}/5...")
    # Generate datasets for the current fold
    train_dataset = tf.data.Dataset.from_tensor_slices((train_val_images[train_idx], train_val_labels[train_idx])).batch(32).shuffle(len(train_idx))
    val_dataset = tf.data.Dataset.from_tensor_slices((train_val_images[val_idx], train_val_labels[val_idx])).batch(32)
    
    # Build and compile model
    model = build_model_with_resnet()
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', Precision(name='precision'), Recall(name='recall'), AUC(name='auc')])
    
    # Fit model
    history = model.fit(train_dataset, epochs=50, validation_data=val_dataset) """

In [15]:
# Cross-validation loop
for fold, (train_idx, val_idx) in enumerate(kf.split(train_val_images)):
    print(f"Training on fold {fold+1}/5...")
    
    # Preprocess and batch the training data with data augmentation
    train_dataset = tf.data.Dataset.from_tensor_slices((train_val_images[train_idx], train_val_labels[train_idx]))
    train_dataset = train_dataset.map(preprocess)  # Apply data augmentation here
    train_dataset = train_dataset.shuffle(len(train_idx)).batch(32).prefetch(tf.data.AUTOTUNE)
    
    # Batch the validation data (without augmentation)
    val_dataset = tf.data.Dataset.from_tensor_slices((train_val_images[val_idx], train_val_labels[val_idx]))
    val_dataset = val_dataset.batch(32).prefetch(tf.data.AUTOTUNE)
    
    # Build and compile the model
    model = build_model_with_resnet()
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', 'precision', 'recall', 'auc'])
    
    # Fit the model on the training data and evaluate on the validation data
    history = model.fit(train_dataset, epochs=50, validation_data=val_dataset)


Training on fold 1/5...
Epoch 1/50


2024-04-10 12:44:24.158052: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2024-04-10 12:44:35.326497: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:6: Filling up shuffle buffer (this may take a while): 1671 of 4069
2024-04-10 12:44:45.330105: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:6: Filling up shuffle buffer (this may take a while): 3383 of 4069
2024-04-10 12:44:49.333836: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:480] Shuffle buffer filled.


128/128 ━━━━━━━━━━━━━━━━━━━━ 45s 140ms/step - accuracy: 0.5798 - auc: 0.5104 - loss: 0.7292 - precision: 0.4007 - recall: 0.2506 - val_accuracy: 0.6257 - val_auc: 0.5078 - val_loss: 0.6628 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/50


2024-04-10 12:45:17.879332: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:6: Filling up shuffle buffer (this may take a while): 1597 of 4069


  2/128 ━━━━━━━━━━━━━━━━━━━━ 11s 93ms/step - accuracy: 0.5859 - auc: 0.5024 - loss: 0.6746 - precision: 0.1429 - recall: 0.0455         

...

128/128 ━━━━━━━━━━━━━━━━━━━━ 40s 113ms/step - accuracy: 0.6153 - auc: 0.5003 - loss: 0.6718 - precision: 0.2915 - recall: 0.0404 - val_accuracy: 0.6238 - val_auc: 0.5195 - val_loss: 0.6671 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 24/50


2024-04-10 12:59:55.618543: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:6: Filling up shuffle buffer (this may take a while): 1603 of 4069


  2/128 ━━━━━━━━━━━━━━━━━━━━ 11s 92ms/step - accuracy: 0.6250 - auc: 0.4548 - loss: 0.6939 - precision: 0.0000e+00 - recall: 0.0000e+00 

2024-04-10 13:00:10.856776: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:480] Shuffle buffer filled.


127/128 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step - accuracy: 0.6285 - auc: 0.5091 - loss: 0.6687 - precision: 0.3408 - recall: 0.0310

In [9]:
# After cross-validation, you might want to retrain your model on all training data and evaluate it on the test set
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).batch(32)
# Make sure to re-build and compile your model

# Assuming 'model' is your final trained model
test_loss, test_acc, test_precision, test_recall, test_auc = model.evaluate(test_dataset)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_acc}")

40/40 ━━━━━━━━━━━━━━━━━━━━ 4s 95ms/step - accuracy: 0.6011 - auc: 0.5366 - loss: 0.6717 - precision: 0.7805 - recall: 0.0053
Test Loss: 0.6659784913063049
Test Accuracy: 0.6092767119407654
